In [7]:
import pandas as pd
import re
import os

# ---- Учитај фајлове ----
df1 = pd.read_csv("IMSBS-1000_heuristic-h5_imbs-iters-500_num-roots-10.csv")  
df2 = pd.read_csv("LIMSBS-1000_imbs-iters-500_num-roots-10_U5_5_5_A1_F3.csv")   #LIMSBS-1000_imbs-iters-500_num-roots-10_U5_5_5_A1_F3.csv

# ---- Извуци 4 броја из имена фајла ----
def extract_numbers(filename):
    # узимамо сва 4 броја из имена: n_m_k_index
    m = re.match(r".*?_(\d+)_(\d+)_(\d+)_(\d+)\.txt\.out", filename)
    if m:
        return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4))
    else:
        return None, None, None, None

for df in [df1, df2]:
    df[['n','m','k','idx']] = df['file'].apply(lambda x: pd.Series(extract_numbers(x)))
    # kreiramo key po prva tri broja
    df['key'] = df[['n','m','k']].astype(str).agg('_'.join, axis=1)

# ---- Spoji podatke po key ----
merged = pd.merge(df1, df2, on='key', suffixes=('_1','_2'))

# ---- Agregiraj po prva tri broja (key) ----
agg = merged.groupby('key').agg({
    'time_1': 'mean',
    'quality_1': 'mean',
    'time_2': 'mean',
    'quality_2': 'mean'
}).reset_index()


# ---- Sačuvaj u CSV ----
agg.to_csv("merged_aggregated.csv", index=False)

# ---- Rezultat ----
agg = agg.rename(columns={"quality_1" : "IMSBS-beta-1000-iter-500-heuristic-h5", "time_1": "IMSBS-time", "quality_2": "LIMSBS-beta-1000-iter-500", "time_2": "LIMSBS-time"})


In [9]:
agg[["IMSBS-beta-1000-iter-500-heuristic-h5", "LIMSBS-beta-1000-iter-500", "IMSBS-time", "LIMSBS-time"]] #.mean()   #[["time_1", "time_2", "quality_1", "quality_2" ]].mean()

,IMSBS-beta-1000-iter-500-heuristic-h5,LIMSBS-beta-1000-iter-500,IMSBS-time,LIMSBS-time
0,10.4,10.7,0.065145,0.108429
1,7.0,8.8,0.137714,0.139030
2,11.4,12.1,0.072177,0.093153
3,7.3,7.0,0.139215,0.166610
4,11.8,12.2,0.060109,0.168830
5,6.9,7.9,0.142968,0.176514
6,11.1,11.0,0.066460,0.061568
7,6.6,6.7,0.132503,0.088200
8,72.9,72.8,0.129203,0.373188
9,61.7,61.8,0.152249,0.344568
